### 작업형 제1유형

#### 1.

In [8]:
import pandas as pd

df = pd.read_csv('../yemoonsaBigdata/datasets/Part3/301_housing.csv')

# 데이터 중 결측치가 있는 경우 해당 데이터의 행을 모두 제거한다.
df = df.dropna(axis=0)
length = int(len(df) * 0.7)

new_df = df.iloc[:length, :]
q1 = new_df['housing_median_age'].quantile(0.25)
print(q1)


19.0


#### 2.

In [25]:
df = pd.read_csv('../yemoonsaBigdata/datasets/Part3/302_worlddata.csv')
year_2000 = df[df['year'] == 2000]
year_2000 = year_2000.T[1].drop(index=['year'])

mean = df[df['year'] == 2000].T.mean().values[0]

print(len(year_2000[year_2000 > mean]))

70


#### 3.

In [34]:
df = pd.read_csv('../yemoonsaBigdata/datasets/Part3/303_titanic.csv')

null = df.isnull().sum()
print(null[null == null.max()].index.values[0])

Age


<br><br><br>

### 작업형 제2유형

In [62]:
import pandas as pd
import numpy as np

X_train = pd.read_csv('../yemoonsaBigdata/datasets/Part3/304_x_train.csv')
y_train = pd.read_csv('../yemoonsaBigdata/datasets/Part3/304_y_train.csv')
X_test = pd.read_csv('../yemoonsaBigdata/datasets/Part3/304_x_test.csv')

num_cols = ['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases']
cat_cols = ['Employment Type', 'GraduateOrNot', 'FrequentFlyer', 'EverTravelledAbroad']
y_cols = ['TravelInsurance']

# 범주형 데이터 전처리
from sklearn.preprocessing import LabelEncoder
X = pd.concat([X_train, X_test])
for col in cat_cols:
    label_encoder = LabelEncoder()
    label_encoder.fit(X[col])
    
    X_train[col] = label_encoder.transform(X_train[col])
    X_test[col] = label_encoder.transform(X_test[col])


# 학습, 검증 데이터셋 
from sklearn.model_selection import train_test_split

X_tr, X_val, y_tr, y_val = train_test_split(X_train[num_cols+cat_cols], y_train[y_cols], test_size=0.3)

# 수치형 데이터셋 전처리
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr[num_cols])

X_tr[num_cols] = scaler.transform(X_tr[num_cols])
X_val[num_cols] = scaler.transform(X_val[num_cols])
X_test[num_cols] = scaler.transform(X_test[num_cols])


# 학습
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
# import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score, roc_auc_score


model_xgb1 = XGBClassifier(n_estimators=50, max_depth=5)
model_xgb1.fit(X_tr, y_tr.values.squeeze())
y_pred = model_xgb1.predict(X_val)
# print(accuracy_score(y_val, y_pred))



# y_pred_proba = model_xgb1.predict_proba(X_val)
# print(y_pred_proba[:, 1])
# print(roc_auc_score(y_val, y_pred_proba[:, 1]))



y_pred_proba = model_xgb1.predict_proba(X_test[num_cols+cat_cols])
# print(y_pred_proba[:, 1])

result = pd.DataFrame({
    'index': X_test['ID'],
    'y_pred': y_pred_proba[:, 1]
})
print(result)

result.to_csv('../yemoonsaBigdata/res/3.csv', index=False)

     index    y_pred
0     1569  0.249780
1     1344  0.980985
2     1429  0.110050
3      896  0.421045
4      101  0.981896
..     ...       ...
492   1376  0.116380
493     87  0.004359
494    287  0.165676
495    337  0.425349
496     92  0.111409

[497 rows x 2 columns]
